In [1]:
library("reticulate")
# use_python("C:/Users/berkc/Miniconda3/envs/R", required = T)
use_python("C:/Users/kapusub/Anaconda3/envs/deep_learning_PIML", required = T)
py_config()

python:         C:/Users/kapusub/Anaconda3/envs/deep_learning_PIML/python.exe
libpython:      C:/Users/kapusub/Anaconda3/envs/deep_learning_PIML/python37.dll
pythonhome:     C:\Users\kapusub\ANACON~1\envs\DEEP_L~2
version:        3.7.7 (default, May  6 2020, 11:45:54) [MSC v.1916 64 bit (AMD64)]
Architecture:   64bit
numpy:          C:\Users\kapusub\ANACON~1\envs\DEEP_L~2\lib\site-packages\numpy
numpy_version:  1.18.1

NOTE: Python version was forced by use_python function

In [2]:
source("sobolGP_poro_np.R")
source("sobolGP_poro_np_upd.R")
source("sobolGPmethods.R")
source("pickfreeze.R")

# Pre-trained model parameters

In [3]:
gp = source_python("GP_pre_train.py")
params <- Gp_phy()
params

1.12**2 * RBF(length_scale=[0.628, 1.97e+03, 1.81, 3.92e+03, 2.53e+08, 6.77e+06, 2.94e+06, 5.06e+06, 1.66e+07, 3.93e+06, 2.42e+08])

In [12]:
# numobs <- c(500,1000,1500,2000,2500)
numobs <- c(100,250,500,750,1000)
# case1 <- c("GP_MC_p_Lake","GP_loss_MC_p_Lake")
# case2 <- c("GP_upd_MC_p_Lake","GP_upd_loss_MC_p_Lake")
case1 <- c("GP_MC_p_Lake")
case2 <- c()
numobs <- c(250)

d <- 11; n <- 100
candidate <- data.frame(matrix(runif(d * 2), nrow = 2))
X1 <- data.frame(matrix(runif(d * n), nrow = n))
X2 <- data.frame(matrix(runif(d * n), nrow = n))

for(j in case1){
    source("sobolGP_poro_np.R")
    GP_file = paste(j, ".py", sep="")    
    cat(GP_file,"\n")
    for(i in numobs){
    
        start_time <- Sys.time()
        #--------------------------------------#
        # Kriging-based Sobol
        #--------------------------------------#
        res <- sobolGP_poro_np(
        model = "GP",
        type="UK",
        MCmethod="sobol2002",
        X1,
        X2,
        nsim = 100,
        conf = 0.95,
        nboot=1,
        sequential = FALSE,
        candidate,
        sequential.tot=FALSE,
        max_iter = 100,
        filename=GP_file,
        nobs = i
        )

        end_time <- Sys.time()
        print(end_time - start_time)
        
        # combine number of observation vector with Sobol indices
#         data <- cbind(i, res$S$mean)
        data <- cbind(i, res$S$mean, t(res$S$ci[1,]), t(res$S$ci[2,]))
        dat <- as.data.frame(data)
        
        # total index
#         dataT <- cbind(i, res$T$mean)
        dataT <- cbind(i, res$T$mean, t(res$T$ci[1,]), t(res$T$ci[2,]))
        datT <- as.data.frame(dataT)
        
        # write to file
        results_dir <- "../../../../results/Lake/"
        # filename <- paste(results_dir, "Case1_", toString(num_obs), ".csv", sep = "")
        # write.csv(x=dat, file=filename)
        # write.table(dat, file=filename, append=T)
        filename <- paste(results_dir, paste(j, "_first_", n,".csv", sep=""), sep = "")
        write.table(dat, filename, sep = ",", col.names = !file.exists(filename), row.names = F, append = T)
        
        filenameT <- paste(results_dir, paste(j, "_tot_", n,".csv", sep=""), sep = "")
        write.table(datT, filenameT, sep = ",", col.names = !file.exists(filenameT), row.names = F, append = T)
    }
}






for(j in case2){
    source("sobolGP_poro_np_upd.R")
    GP_file = paste(j, ".py", sep="")    
    cat(GP_file,"\n")
    for(i in numobs){
    
        start_time <- Sys.time()
        #--------------------------------------#
        # Kriging-based Sobol
        #--------------------------------------#
        res <- sobolGP_poro_np_upd(
        model = "GP",
        type="UK",
        MCmethod="sobol2002",
        X1,
        X2,
        nsim = 100,
        conf = 0.95,
        nboot=1,
        sequential = FALSE,
        candidate,
        sequential.tot=FALSE,
        max_iter = 100,
        filename=GP_file,
        params,
        nobs = i
        )
        
        end_time <- Sys.time()
        print(end_time - start_time)
        
        # combine number of observation vector with Sobol indices
#         data <- cbind(i, res$S$mean)
        data <- cbind(i, res$S$mean, t(res$S$ci[1,]), t(res$S$ci[2,]))
        dat <- as.data.frame(data)
        
        # total index
#         dataT <- cbind(i, res$T$mean)
        dataT <- cbind(i, res$T$mean, t(res$T$ci[1,]), t(res$T$ci[2,]))
        datT <- as.data.frame(dataT)
        
        # write to file
        results_dir <- "../../../../results/Lake/"
        # filename <- paste(results_dir, "Case1_", toString(num_obs), ".csv", sep = "")
        # write.csv(x=dat, file=filename)
        # write.table(dat, file=filename, append=T)
        filename <- paste(results_dir, paste(j, "_first_", n,"_10restart.csv", sep=""), sep = "")
        write.table(dat, filename, sep = ",", col.names = !file.exists(filename), row.names = F, append = T)
        
        filenameT <- paste(results_dir, paste(j, "_tot_", n,"_10restart.csv", sep=""), sep = "")
        write.table(datT, filenameT, sep = ",", col.names = !file.exists(filenameT), row.names = F, append = T)
    }
}

GP_MC_p_Lake.py 
Time difference of 2.080971 mins


In [34]:
res$S

0.2024666,0.7050228
0.2383298,0.3102915
-0.3031855,0.1109655
0.6008208,1.0840103
0.2383298,0.3102915
NA,NA
